In [ ]:
#Cargamos librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from imblearn.over_sampling import SMOTE
from collections import Counter
import sklearn
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression 
from sklearn.feature_selection import chi2 
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [ ]:
#Leemos el nuestra base de datos incendios_fin_base4.csv

In [2]:
dfInc = pd.read_csv("incendios_fin_base4.csv", dtype = {'COD_AEMET': object}, encoding = "UTF-8")

In [3]:
dfInc.isna().sum()

id                    0
latitude              0
longitude             0
idcomunidad           0
comunidad             0
idprovincia           0
provincia             0
COD_AEMET             0
municipio             0
zona                  0
altitud               0
poblacion_muni        0
superficie_muni       0
id_zona_meteo         0
zona_meteo            0
NATCODE               0
fecha                 0
tmax                  0
tmin                  0
racha                 0
prec                  0
velmedia              0
dir                   0
fe_month              0
superficie            0
superficie_quemada    0
dtype: int64

In [4]:
# display updated DataFrame
dfInc.head()

,id,latitude,longitude,idcomunidad,comunidad,idprovincia,provincia,COD_AEMET,municipio,zona,...,fecha,tmax,tmin,racha,prec,velmedia,dir,fe_month,superficie,superficie_quemada
0,2003010036,42.847480,-2.498224,16,País Vasco,1,Araba/Álava,01001,Alegría-Dulantzi,Noroeste,...,2003-09-21,29.3,11.4,7.8,0,1.1,S,9,5.23,Incendio_Mediano
1,2001010030,43.010649,-2.926375,16,País Vasco,1,Araba/Álava,01002,Amurrio,Noroeste,...,2001-08-27,31.9,16.3,8.3,0,1.7,NE,8,16.61,Incendio_Mediano
2,2004010037,43.001667,-2.987731,16,País Vasco,1,Araba/Álava,01002,Amurrio,Noroeste,...,2004-09-01,28.0,11.6,7.8,1,1.4,O,9,3.66,Incendio_Pequeno
3,2002010005,43.010649,-2.926375,16,País Vasco,1,Araba/Álava,01002,Amurrio,Noroeste,...,2002-01-29,16.0,9.6,6.9,0,2.2,S,1,4.00,Incendio_Pequeno
4,2001010027,43.010649,-2.926375,16,País Vasco,1,Araba/Álava,01002,Amurrio,Noroeste,...,2001-08-22,24.8,14.7,6.7,0,0.6,NE,8,4.13,Incendio_Pequeno


In [ ]:
#Variable objetivo

In [5]:
dfInc = dfInc.rename(columns={'superficie_quemada': 'supQuemada'})

In [6]:
dfInc['supQuemada'].value_counts()

Incendio_Pequeno       45078
Incendio_Mediano       21320
Conato                 10341
Incendio_Grande         3013
Incendio_Muy_Grande      659
Gran_Incendio            358
Name: supQuemada, dtype: int64

In [7]:
dfInc['supQuemada'].value_counts(normalize=True)

Incendio_Pequeno       0.558110
Incendio_Mediano       0.263963
Conato                 0.128032
Incendio_Grande        0.037304
Incendio_Muy_Grande    0.008159
Gran_Incendio          0.004432
Name: supQuemada, dtype: float64

In [8]:
datos = dfInc.copy()

In [ ]:
#Variables de nuestra base de datos

In [9]:
datos.columns

Index(['id', 'latitude', 'longitude', 'idcomunidad', 'comunidad',
       'idprovincia', 'provincia', 'COD_AEMET', 'municipio', 'zona', 'altitud',
       'poblacion_muni', 'superficie_muni', 'id_zona_meteo', 'zona_meteo',
       'NATCODE', 'fecha', 'tmax', 'tmin', 'racha', 'prec', 'velmedia', 'dir',
       'fe_month', 'superficie', 'supQuemada'],
      dtype='object')

In [10]:
#Pasamos a número las variables categoricas
datos.dir.replace(('NE','C','SE','SO','N','S','O'),
                      (1,2,3,4,5,6,7),inplace=True)

datos.supQuemada.replace(('Conato', 'Incendio_Pequeno', 'Incendio_Mediano', 'Incendio_Grande', 'Incendio_Muy_Grande','Gran_Incendio'),
                     (1,2,3,4,5,6),inplace=True)

datos.zona.replace(('Noroeste', 'Interior', 'Mediterráneo', 'Canarias'), 
                   (1,2,3,4),inplace=True)


In [11]:
#Pasamos a númerica la variable municipio
a = datos.municipio.unique()
b = range(1,len(a) + 1)
dict_from_list = dict(zip(a, b))
datos['idmunicipio'] = datos.municipio.map(dict_from_list)

In [12]:
#Feature engineering
##distancia
datos["dist"] = np.sqrt(pow(datos["longitude"],2)+pow(datos["latitude"],2))

##densidad de poblacion
datos["densPob"] = datos["poblacion_muni"]/datos["superficie_muni"]

##sensacion termica por frio
#datos["sensaTerF"]= 13.1267 + (0.6215*(datos["tmax"]))- pow((11.37*(datos["racha"])),0.16) + pow((0.3965*(datos["tmax"])*(datos["racha"])),0.16)

##trimestres
conditions = [
    (datos['fe_month'] <= 3),
    (datos['fe_month'] > 3) & (datos['fe_month'] <= 6),
    (datos['fe_month'] > 6) & (datos['fe_month'] < 10),
    (datos['fe_month'] >= 10),
]
values = [1, 2, 3, 4]
datos['trimest'] = np.select(conditions, values)

##Número de Beaufort con racha
conditions = [
    (datos['racha'] < 2),
    (datos['racha'] >= 2) & (datos['racha'] < 6),
    (datos['racha'] >= 6) & (datos['racha'] < 12),
    (datos['racha'] >= 12) & (datos['racha'] < 20),
    (datos['racha'] >= 20) & (datos['racha'] < 29),
    (datos['racha'] >= 29) & (datos['racha'] < 39),
    (datos['racha'] >= 39) & (datos['racha'] < 50),
]
values = [0,1,2,3,4,5,6]
datos['beauVelmed'] = np.select(conditions, values)

##Número de Beaufort con racha
conditions = [
    (datos['racha'] < 2),
    (datos['racha'] >= 2) & (datos['racha'] < 6),
    (datos['racha'] >= 6) & (datos['racha'] < 12),
    (datos['racha'] >= 12) & (datos['racha'] < 20),
    (datos['racha'] >= 20) & (datos['racha'] < 29),
    (datos['racha'] >= 29) & (datos['racha'] < 39),
    (datos['racha'] >= 39) & (datos['racha'] < 50),
]
values = [0,1,2,3,4,5,6]
datos['beauRacha'] = np.select(conditions, values)

##temperatura maxima categorica
conditions = [
    (datos['tmax'] <= -10),
    (datos['tmax'] > -10) & (datos['tmax'] <= 0),
    (datos['tmax'] > 0) & (datos['tmax'] <= 10),
    (datos['tmax'] > 10) & (datos['tmax'] <= 20),
    (datos['tmax'] > 20) & (datos['tmax'] <= 30),
    (datos['tmax'] > 30) & (datos['tmax'] <= 40),
    (datos['tmax'] > 40),
]
values = [0,1,2,3,4,5,6]
datos['tmaxCat'] = np.select(conditions, values)

#temperatura minima categorica
conditions = [
    (datos['tmin'] <= -10),
    (datos['tmin'] > -10) & (datos['tmin'] <= 0),
    (datos['tmin'] > 0) & (datos['tmin'] <= 10),
    (datos['tmin'] > 10) & (datos['tmin'] <= 20),
    (datos['tmin'] > 20) & (datos['tmin'] <= 30),
    (datos['tmin'] > 30) & (datos['tmin'] <= 40),
    (datos['tmin'] > 40),
]
values = [0,1,2,3,4,5,6]
datos['tminCat'] = np.select(conditions, values)

In [13]:
def eliminarCol(x): return datos.drop(x, axis=1, inplace=True)

In [14]:
#Eliminamos la variables que no vamos a utilizar
eliminarCol('comunidad')
eliminarCol('provincia')
eliminarCol('municipio')
eliminarCol('fecha')
eliminarCol('superficie')
eliminarCol('zona_meteo')

In [15]:
datosML = datos.loc[:, datos.columns != 'supQuemada']

In [16]:
datosML.head()

,id,latitude,longitude,idcomunidad,idprovincia,COD_AEMET,zona,altitud,poblacion_muni,superficie_muni,...,dir,fe_month,idmunicipio,dist,densPob,trimest,beauVelmed,beauRacha,tmaxCat,tminCat
0,2003010036,42.847480,-2.498224,16,1,01001,1,513,2935,1994.5872,...,6,9,1,42.920248,1.471482,3,2,2,4,3
1,2001010030,43.010649,-2.926375,16,1,01002,1,208,10264,9649.5300,...,1,8,2,43.110087,1.063679,3,2,2,5,3
2,2004010037,43.001667,-2.987731,16,1,01002,1,208,10264,9649.5300,...,7,9,2,43.105335,1.063679,3,2,2,4,3
3,2002010005,43.010649,-2.926375,16,1,01002,1,208,10264,9649.5300,...,6,1,2,43.110087,1.063679,1,2,2,3,2
4,2001010027,43.010649,-2.926375,16,1,01002,1,208,10264,9649.5300,...,1,8,2,43.110087,1.063679,3,2,2,4,3


In [17]:
# Normalización:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(datosML)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.columns = datosML.columns.values

In [18]:
df_normalized['supQuemada'] = datos['supQuemada']

In [19]:
df_normalized.head()

,id,latitude,longitude,idcomunidad,idprovincia,COD_AEMET,zona,altitud,poblacion_muni,superficie_muni,...,fe_month,idmunicipio,dist,densPob,trimest,beauVelmed,beauRacha,tmaxCat,tminCat,supQuemada
0,0.137933,0.943081,0.699894,0.882353,0.0,0.00000,0.0,0.215703,0.000878,0.011078,...,0.727273,0.000000,0.877416,0.008649,0.666667,0.333333,0.333333,0.666667,0.75,3
1,0.000002,0.953245,0.680593,0.882353,0.0,0.00002,0.0,0.086957,0.003076,0.054829,...,0.636364,0.000176,0.892472,0.006247,0.666667,0.333333,0.333333,0.833333,0.75,3
2,0.206899,0.952685,0.677827,0.882353,0.0,0.00002,0.0,0.086957,0.003076,0.054829,...,0.727273,0.000176,0.892095,0.006247,0.666667,0.333333,0.333333,0.666667,0.75,2
3,0.068966,0.953245,0.680593,0.882353,0.0,0.00002,0.0,0.086957,0.003076,0.054829,...,0.000000,0.000176,0.892472,0.006247,0.000000,0.333333,0.333333,0.500000,0.50,2
4,0.000002,0.953245,0.680593,0.882353,0.0,0.00002,0.0,0.086957,0.003076,0.054829,...,0.636364,0.000176,0.892472,0.006247,0.666667,0.333333,0.333333,0.666667,0.75,2


In [20]:
X = df_normalized.loc[:, df_normalized.columns != 'supQuemada']
y = df_normalized['supQuemada']

In [21]:
df_normalized.isna().sum()

id                 0
latitude           0
longitude          0
idcomunidad        0
idprovincia        0
COD_AEMET          0
zona               0
altitud            0
poblacion_muni     0
superficie_muni    0
id_zona_meteo      0
NATCODE            0
tmax               0
tmin               0
racha              0
prec               0
velmedia           0
dir                0
fe_month           0
idmunicipio        0
dist               0
densPob            0
trimest            0
beauVelmed         0
beauRacha          0
tmaxCat            0
tminCat            0
supQuemada         0
dtype: int64

**Seleccion de variables**

In [ ]:
#Correlación de la variables/SupQuemada

In [22]:
corr = abs(df_normalized.corr())
corr[['supQuemada']].sort_values(by = 'supQuemada',ascending = False).style.background_gradient()

,supQuemada
supQuemada,1.000000
id,0.108105
tmax,0.076227
tmaxCat,0.071207
tmin,0.059425
tminCat,0.057226
trimest,0.046444
fe_month,0.046090
idcomunidad,0.040371
NATCODE,0.040113


In [ ]:
#Selección de las mejores características (16,11 y 6)

In [23]:
S_chi5 = SelectKBest(chi2, k = 16)
X_chi5 = S_chi5.fit_transform(df_normalized, df_normalized['supQuemada'])

variables = list(np.asarray(list(df_normalized))[S_chi5.get_support()])
variables.sort()
print(variables)

['NATCODE', 'altitud', 'fe_month', 'id', 'id_zona_meteo', 'idcomunidad', 'prec', 'racha', 'supQuemada', 'superficie_muni', 'tmax', 'tmaxCat', 'tmin', 'tminCat', 'trimest', 'zona']


In [24]:
S_chi5 = SelectKBest(chi2, k = 11)
X_chi5 = S_chi5.fit_transform(df_normalized, df_normalized['supQuemada'])

variables = list(np.asarray(list(df_normalized))[S_chi5.get_support()])
variables.sort()
print(variables)

['NATCODE', 'fe_month', 'id', 'idcomunidad', 'prec', 'supQuemada', 'tmax', 'tmaxCat', 'tminCat', 'trimest', 'zona']


In [25]:
S_chi5 = SelectKBest(chi2, k = 6)
X_chi5 = S_chi5.fit_transform(df_normalized, df_normalized['supQuemada'])

variables = list(np.asarray(list(df_normalized))[S_chi5.get_support()])
variables.sort()
print(variables)

['fe_month', 'id', 'supQuemada', 'tmax', 'trimest', 'zona']


In [ ]:
#Correlación

In [26]:
correlation_mat = df_normalized.corr()
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
strong_pairs = sorted_pairs[abs(sorted_pairs) > 0.5]
strong_pairs.head(35)

zona            dist             -0.879047
dist            zona             -0.879047
latitude        zona             -0.833552
zona            latitude         -0.833552
longitude       zona              0.517473
zona            longitude         0.517473
densPob         poblacion_muni    0.573596
poblacion_muni  densPob           0.573596
tmaxCat         tminCat           0.670295
tminCat         tmaxCat           0.670295
beauRacha       velmedia          0.696671
beauVelmed      velmedia          0.696671
velmedia        beauVelmed        0.696671
                beauRacha         0.696671
tminCat         tmax              0.710600
tmax            tminCat           0.710600
racha           velmedia          0.711512
velmedia        racha             0.711512
tmin            tmaxCat           0.725270
tmaxCat         tmin              0.725270
tmax            tmin              0.766976
tmin            tmax              0.766976
tminCat         tmin              0.916278
tmin       

Modelos

In [27]:
df_normalized.columns

Index(['id', 'latitude', 'longitude', 'idcomunidad', 'idprovincia',
       'COD_AEMET', 'zona', 'altitud', 'poblacion_muni', 'superficie_muni',
       'id_zona_meteo', 'NATCODE', 'tmax', 'tmin', 'racha', 'prec', 'velmedia',
       'dir', 'fe_month', 'idmunicipio', 'dist', 'densPob', 'trimest',
       'beauVelmed', 'beauRacha', 'tmaxCat', 'tminCat', 'supQuemada'],
      dtype='object')

In [28]:
variables = ['altitud', 'racha', 'densPob', 'idcomunidad', 'superficie_muni', 'prec', 'tmaxCat', 'trimest', 'dist']
variables = df_normalized[variables]

In [29]:
correlation_mat = variables.corr()
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
strong_pairs = sorted_pairs[abs(sorted_pairs) > 0.5]
strong_pairs.head(35)

altitud          altitud            1.0
superficie_muni  superficie_muni    1.0
tmaxCat          tmaxCat            1.0
prec             prec               1.0
idcomunidad      idcomunidad        1.0
densPob          densPob            1.0
racha            racha              1.0
trimest          trimest            1.0
dist             dist               1.0
dtype: float64

In [ ]:
#Selección de train y test

In [30]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(variables,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify = y)

In [ ]:
#Modelos

In [31]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print ("Distribution of class labels before resampling {}".format(Counter(y_train)))
print ("Distribution of class labels after resampling {}".format(Counter(y_train_res)))

Distribution of class labels before resampling Counter({2: 36062, 3: 17056, 1: 8273, 4: 2410, 5: 527, 6: 287})
Distribution of class labels after resampling Counter({2: 36062, 1: 36062, 3: 36062, 5: 36062, 4: 36062, 6: 36062})


In [35]:
use_label_encoder=False
models = {
    "neighbors": KNeighborsClassifier(),
    "histboosting": HistGradientBoostingClassifier(),
    "histboosting0.2": HistGradientBoostingClassifier(learning_rate=0.25),
    "forest300,50": RandomForestClassifier(n_estimators=300, max_depth=10),#,class_weight="balanced"),
    "Gradientboosting" : GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(n_estimators = 100)
}
for name, model in models.items():
    scores = cross_val_score(model, X_train_res, y_train_res, cv=10)
    print(f"{name} accuracy => {np.mean(scores)}")
    print(scores.mean())
    print(scores.std())

neighbors accuracy => 0.7351322946388008
0.7351322946388008
0.015157316978774024
histboosting accuracy => 0.6014832455776769
0.6014832455776769
0.03530683409834005
histboosting0.2 accuracy => 0.6637557434307847
0.6637557434307847
0.04236191580995601
forest300,50 accuracy => 0.5407818261368542
0.5407818261368542
0.012642421994648203
Gradientboosting accuracy => 0.4268992865556049
0.4268992865556049
0.014987314445927663


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:58:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:00:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[15:08:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:09:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:11:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost accuracy => 0.6390298768080507
0.6390298768080507
0.04630573917910128


NEIGHBORS

In [36]:
neighbors = KNeighborsClassifier().fit(X_train_res, y_train_res)
print(neighbors.score(X_train_res, y_train_res))
print(neighbors.score(X_test, y_test))

0.8253563307636848
0.32889686764887954


HIST GRADIENT BOOSTING

In [37]:
hist = HistGradientBoostingClassifier(learning_rate=0.2).fit(X_train_res, y_train_res)
print(hist.score(X_train_res, y_train_res))
print(hist.score(X_test, y_test))

0.6914018449706986
0.45196236226321657


RANDOM FOREST

In [38]:
forest = RandomForestClassifier(n_estimators=300, max_depth=10).fit(X_train_res, y_train_res)
print(forest.score(X_train_res, y_train_res))
print(forest.score(X_test, y_test))

0.5660298005287191
0.2873591680079237


GRADIENT BOOSTING

In [39]:
grad = GradientBoostingClassifier().fit(X_train_res, y_train_res)
print(grad.score(X_train_res, y_train_res))
print(grad.score(X_test, y_test))

0.43145138927402804
0.30110189426767364


XGBOOST

In [34]:
use_label_encoder=False
xgb = XGBClassifier(n_estimators = 300).fit(X_train_res, y_train_res)
print(xgb.score(X_train_res, y_train_res))
print(xgb.score(X_test, y_test))

[17:01:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7901761780637051
0.5040856753745202


GRID SEARCH

In [ ]:
use_label_encoder=False
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators'  : [300, 500],
              'max_depth'     : [3, 5],
            }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = XGBClassifier(random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train_res, y = y_train_res)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

CROSS VALIDATION ON SELECTED MODEL (XGBOOST)

In [41]:
use_label_encoder=False
cross_val_xgb = cross_val_score(xgb, X_train_res, y_train_res, cv=10)
print(cross_val_xgb)
print(cross_val_xgb.mean())
print(cross_val_xgb.std())

C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:24:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:28:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[15:59:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:04:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\cregueir\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:09:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.55758388 0.63892227 0.70430281 0.72311319 0.73147849 0.74515876
 0.74631418 0.75001155 0.73623885 0.74511254]
0.707823651586881
0.05914240607468446


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

EXPORT MODEL

In [36]:
import pickle

In [37]:
pickle.dump(xgb, open("pima.pickle.dat", "wb"))